In [1]:
import pandas as pd
import numpy  as np
import json
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.simplefilter("ignore")
hfont = {'fontname':'Monaco'}
hfont = {'fontsize':'24'}

# Making the plots standard 
%matplotlib inline
plt.rcParams["figure.figsize"] = [16, 5]

In [2]:
#import the training and test pickles 
train_df = pd.read_pickle('train_df.pickle')
test_df = pd.read_pickle('test_df.pickle')

#get the training std 
trainrtt_mean = train_df['last_rtt'].mean()
trainrtt_std = train_df['last_rtt'].std()

In [3]:
cols_dropped = ['date','last_rtt','normalizzed_rtt','src_names']

X_train = train_df.drop(columns=cols_dropped, axis=1)
y_train = train_df['normalizzed_rtt']

X_test = test_df.drop(columns=cols_dropped, axis=1)
y_test = test_df['normalizzed_rtt'] 

In [4]:
# Using Prophet for time series forecasting
# https://www.youtube.com/watch?v=j_KuqOktXZM&list=PL6UJrXaOPPcEesMFPaieQJwnBvKiRH2tT&index=2
# https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html
from prophet import Prophet 
from sklearn.metrics import mean_squared_error

# rename columns names for Prophet
prophet_train = test_df.copy()
prophet_train = prophet_train.rename(columns={'date':'ds', 'normalizzed_rtt':'y'})

# Initialize and Fit the Prophet Model
model = Prophet()

#adding the regressors
features = X_test.columns

for feature in features:
    model.add_regressor(feature)


# fit the model to the data
model.fit(prophet_train)

# Make In-Sample Predictions
in_sample_forecast = model.predict(prophet_train)

#extract the test data from the test_df
prophet_test = test_df.rename(columns={'date':'ds', 'normalizzed_rtt':'y'})
y_test = prophet_test['y']

# calculate the mean squared error and standard deviation
sample_size = len(y_test)
numerator = abs(in_sample_forecast['yhat'] - y_test)**2 
denominator = sample_size - 1

#loss function for Prophet
rmse_prophet = np.sqrt(mean_squared_error(y_test, in_sample_forecast['yhat']))
drmse_prophet = rmse_prophet * trainrtt_std
print(f'RMSE for Prophet: ', rmse_prophet, 'denormalised rmse: ', drmse_prophet)

12:58:41 - cmdstanpy - INFO - Chain [1] start processing
12:59:02 - cmdstanpy - INFO - Chain [1] done processing
12:59:02 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
13:00:01 - cmdstanpy - INFO - Chain [1] start processing
13:22:55 - cmdstanpy - INFO - Chain [1] done processing


RMSE for Prophet:  1.394979396803665 denormalised rmse:  13.031969584032549
